In [ ]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("recommandation").getOrCreate()

22/12/31 00:13:17 WARN Utils: Your hostname, david0121-MS-7C02 resolves to a loopback address: 127.0.1.1; using 192.168.2.120 instead (on interface enp34s0)
22/12/31 00:13:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/12/31 00:13:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df_train = spark.read.csv('./train_student.csv',header=True,inferSchema=True)


In [3]:
df_train.printSchema()

root
 |-- item: string (nullable = true)
 |-- user: string (nullable = true)
 |-- rating: integer (nullable = true)



In [4]:
from pyspark.sql.functions import format_number, lit
df_train = df_train.withColumn('rating', df_train.rating.cast('float'))

In [5]:
df_train.printSchema()

root
 |-- item: string (nullable = true)
 |-- user: string (nullable = true)
 |-- rating: float (nullable = true)



In [6]:
df_train.describe().show()

+-------+--------------------+--------------------+------------------+
|summary|                item|                user|            rating|
+-------+--------------------+--------------------+------------------+
|  count|               83798|               83798|             83798|
|   mean|2.0769135928283918E9|                null| 4.359018114990811|
| stddev|2.0710778913333938E9|                null|0.9940809586163019|
|    min|          0439893577|A012468118FTQAINE...|               1.0|
|    max|          B00LBI9BKA|       AZZYW4YOE1B6E|               5.0|
+-------+--------------------+--------------------+------------------+



In [7]:
df_train.head(10)

[Row(item='B008H54GVE', user='A2OIMJEGOCTQ87', rating=4.0),
 Row(item='B001NLISDG', user='A38KQZS5M1A8T8', rating=4.0),
 Row(item='B007S3S8HO', user='A1F5O1USOUOOXI', rating=3.0),
 Row(item='B008G6OOHA', user='A1FTZ5LLEX7NCM', rating=4.0),
 Row(item='B00A9JNR8E', user='A1R2JUOGIYH6HO', rating=4.0),
 Row(item='B00000JH3R', user='A2X7C89I7YRX1O', rating=5.0),
 Row(item='B000RXPU0U', user='A24BSKCWXC4M6D', rating=5.0),
 Row(item='B00DWXUYN0', user='A3JX1D26WFEXOS', rating=5.0),
 Row(item='B000E9DPVI', user='A2IWHA1XEYSPD6', rating=5.0),
 Row(item='B001EB9F3C', user='A2340917M1HHZ3', rating=4.0)]

Useful function: indexing ID(string) to numerical index

In [8]:
from pyspark.ml.feature import StringIndexer, IndexToString
from pyspark.ml import Pipeline


In [9]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").setHandleInvalid("keep").fit(df_train) for column in list(set(df_train.columns)-set(['rating'])) ]
pipeline = Pipeline(stages=indexers)
df_ID = pipeline.fit(df_train).transform(df_train)

In [10]:
df_ID.head(10)

22/12/31 00:13:27 WARN DAGScheduler: Broadcasting large task binary with size 1185.7 KiB


[Row(item='B008H54GVE', user='A2OIMJEGOCTQ87', rating=4.0, item_index=5709.0, user_index=1292.0),
 Row(item='B001NLISDG', user='A38KQZS5M1A8T8', rating=4.0, item_index=1849.0, user_index=4940.0),
 Row(item='B007S3S8HO', user='A1F5O1USOUOOXI', rating=3.0, item_index=937.0, user_index=669.0),
 Row(item='B008G6OOHA', user='A1FTZ5LLEX7NCM', rating=4.0, item_index=10888.0, user_index=107.0),
 Row(item='B00A9JNR8E', user='A1R2JUOGIYH6HO', rating=4.0, item_index=1979.0, user_index=1210.0),
 Row(item='B00000JH3R', user='A2X7C89I7YRX1O', rating=5.0, item_index=979.0, user_index=4764.0),
 Row(item='B000RXPU0U', user='A24BSKCWXC4M6D', rating=5.0, item_index=6385.0, user_index=2866.0),
 Row(item='B00DWXUYN0', user='A3JX1D26WFEXOS', rating=5.0, item_index=9620.0, user_index=8104.0),
 Row(item='B000E9DPVI', user='A2IWHA1XEYSPD6', rating=5.0, item_index=420.0, user_index=11230.0),
 Row(item='B001EB9F3C', user='A2340917M1HHZ3', rating=4.0, item_index=51.0, user_index=10687.0)]

In [11]:
(train, test) = df_ID.randomSplit([0.8, 0.2], seed = 2022)

Useful function: Construct Binary data \\
adding 1 for watched and 0 for not watched

In [12]:
def get_binary_data(ratings):
    ratings = df_ID.withColumn('binary', lit(1))
    userIds = df_ID.select("user_index").distinct()
    itemIds = df_ID.select("item_index").distinct()

    user_item = userIds.crossJoin(itemIds).join(ratings, ['user_index', 'item_index'], "left")
    user_item = user_item.select(['user_index', 'item_index', 'binary']).fillna(0)
    return user_item

user_item = get_binary_data(df_ID)

In [13]:
user_item.show()

22/12/31 00:13:29 WARN DAGScheduler: Broadcasting large task binary with size 1185.1 KiB


+----------+----------+------+
|user_index|item_index|binary|
+----------+----------+------+
|    5776.0|     305.0|     0|
|    5776.0|    2815.0|     0|
|    5776.0|    9753.0|     0|
|    5776.0|     692.0|     0|
|    5776.0|     299.0|     0|
|    5776.0|     596.0|     0|
|    5776.0|     496.0|     0|
|    5776.0|    5776.0|     0|
|    5776.0|    2734.0|     0|
|    5776.0|    1051.0|     0|
|    5776.0|    2862.0|     0|
|    5776.0|     769.0|     0|
|    5776.0|    6433.0|     0|
|    5776.0|    5360.0|     0|
|    5776.0|     934.0|     0|
|    5776.0|    5983.0|     0|
|    5776.0|    6454.0|     0|
|    5776.0|   11757.0|     0|
|    5776.0|     558.0|     0|
|    5776.0|    7313.0|     0|
+----------+----------+------+
only showing top 20 rows



22/12/31 00:13:31 WARN DAGScheduler: Broadcasting large task binary with size 1369.6 KiB


Useful Function: pyspark recommandation system ML model \\
Remember to set coldStartStrategy="nan" to make sure model not ignore null data.

Spark allows users to set the coldStartStrategy parameter to “drop” in order to drop any rows in the DataFrame of predictions that contain NaN values. The evaluation metric will then be computed over the non-NaN data and will be valid. Usage of this parameter is illustrated in the example below.

In [14]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
# Create ALS model
als = ALS(
        rank=10,
        maxIter=10,
        regParam=0.5,
        userCol="user_index", 
        itemCol="item_index",
        ratingCol="rating", 
        nonnegative = True, 
        implicitPrefs = False,
        coldStartStrategy="nan"
)
NAN = 4

In [20]:
# Import the requisite packages
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")

Useful function: fill value to NaN part \\
example : \\
a = a.na.fill(value=999)

In [21]:
#Fit cross validator to the 'train' dataset
model = als.fit(train)
#model = cv.fit(train)
#best_model = model.bestModel

# View the predictions
test_predictions = model.transform(test)
#test_predictions = best_model.transform(test)
# remember to fix nan in prediction
test_predictions=test_predictions.na.fill(value=NAN)

RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

22/12/31 00:17:21 WARN DAGScheduler: Broadcasting large task binary with size 1357.1 KiB
22/12/31 00:17:22 WARN DAGScheduler: Broadcasting large task binary with size 1360.1 KiB
22/12/31 00:17:22 WARN DAGScheduler: Broadcasting large task binary with size 1361.6 KiB
22/12/31 00:17:22 WARN DAGScheduler: Broadcasting large task binary with size 1362.9 KiB
22/12/31 00:17:22 WARN DAGScheduler: Broadcasting large task binary with size 1361.8 KiB
22/12/31 00:17:23 WARN DAGScheduler: Broadcasting large task binary with size 1363.1 KiB
22/12/31 00:17:23 WARN DAGScheduler: Broadcasting large task binary with size 1363.9 KiB
22/12/31 00:17:23 WARN DAGScheduler: Broadcasting large task binary with size 1367.0 KiB
22/12/31 00:17:23 WARN DAGScheduler: Broadcasting large task binary with size 1368.4 KiB
22/12/31 00:17:23 WARN DAGScheduler: Broadcasting large task binary with size 1369.7 KiB
22/12/31 00:17:23 WARN DAGScheduler: Broadcasting large task binary with size 1371.1 KiB
22/12/31 00:17:23 WAR

1.2304833165796725


22/12/31 00:17:31 WARN DAGScheduler: Broadcasting large task binary with size 1443.9 KiB


In [22]:
df_public = spark.read.csv('./test_public.csv',header=True,inferSchema=True)
df_public_ID = pipeline.fit(df_public).transform(df_public)

In [23]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="rating",predictionCol="prediction")
predictions=model.transform(df_public_ID)

# remember to fix nan in prediction
predictions=predictions.na.fill(value=NAN)

rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show()

22/12/31 00:17:31 WARN DAGScheduler: Broadcasting large task binary with size 1345.7 KiB
22/12/31 00:17:31 WARN DAGScheduler: Broadcasting large task binary with size 1401.7 KiB
22/12/31 00:17:31 WARN DAGScheduler: Broadcasting large task binary with size 1400.4 KiB
22/12/31 00:17:32 WARN DAGScheduler: Broadcasting large task binary with size 1575.0 KiB
22/12/31 00:17:34 WARN DAGScheduler: Broadcasting large task binary with size 1594.9 KiB
22/12/31 00:17:36 WARN DAGScheduler: Broadcasting large task binary with size 1595.8 KiB


RMSE=1.2254533350717227


22/12/31 00:17:36 WARN DAGScheduler: Broadcasting large task binary with size 1400.4 KiB
22/12/31 00:17:36 WARN DAGScheduler: Broadcasting large task binary with size 1401.7 KiB
22/12/31 00:17:36 WARN DAGScheduler: Broadcasting large task binary with size 1346.1 KiB
22/12/31 00:17:37 WARN DAGScheduler: Broadcasting large task binary with size 1576.0 KiB


+----------+--------------+------+----------+----------+----------+
|      item|          user|rating|item_index|user_index|prediction|
+----------+--------------+------+----------+----------+----------+
|B000SQTJJO|A31BD4RXCON7QO|     4|     148.0|    7634.0| 3.4450731|
|B000SQTJJO|A2TTHN1UMO82VY|     5|     148.0|     329.0| 3.5525923|
|B000SQTJJO|A3LUYUZNKG378S|     5|     148.0|   12556.0| 4.1586866|
|B000SQTJJO|A1RF9YK4BK5TRH|     3|     148.0|     206.0|  3.292471|
|B000SQTJJO|A265B1IZE5RVG6|     2|     148.0|     429.0| 3.5757942|
|B000SQTJJO| A2W34ZSDBOPC6|     4|     148.0|   11665.0|  4.175356|
|B000SQTJJO|A3DYBTW1TEZL3M|     4|     148.0|    1725.0| 4.1937737|
|B000SQTJJO|A2HELIKP5RV27F|     4|     148.0|     695.0| 3.5518086|
|B000SQTJJO|A3J2YU2D9BH2J7|     4|     148.0|    8079.0|  3.588092|
|B000SQTJJO|A1YCWZWOXLUAY5|     5|     148.0|   14912.0|       4.0|
|B000SQTJJO| AQFFA5JFDLQRS|     1|     148.0|   13656.0| 2.9760876|
|B000SQTJJO|A2M0RGVSV6YCMZ|     4|     148.0|   

22/12/31 00:17:39 WARN DAGScheduler: Broadcasting large task binary with size 1591.6 KiB


In [24]:
df_testall = spark.read.csv('./test_all.csv',header=True,inferSchema=True)


In [25]:
df_test_ID = pipeline.fit(df_testall).transform(df_testall)

In [26]:
df_test_ID.printSchema()

root
 |-- item: string (nullable = true)
 |-- user: string (nullable = true)
 |-- item_index: double (nullable = false)
 |-- user_index: double (nullable = false)



In [27]:
df_test_ID.describe().show()

22/12/31 00:17:40 WARN DAGScheduler: Broadcasting large task binary with size 1356.2 KiB


+-------+--------------------+--------------------+------------------+-----------------+
|summary|                item|                user|        item_index|       user_index|
+-------+--------------------+--------------------+------------------+-----------------+
|  count|               83799|               83799|             83799|            83799|
|   mean|2.1813556526264625E9|                null|3844.3237150801324|8276.219012160049|
| stddev|2.1882518602245975E9|                null| 3647.800641218535|6463.700444398778|
|    min|          0439893577|A012468118FTQAINE...|               0.0|              0.0|
|    max|          B00LBI9BKA|       AZZYW4YOE1B6E|           11824.0|          19127.0|
+-------+--------------------+--------------------+------------------+-----------------+



22/12/31 00:17:41 WARN DAGScheduler: Broadcasting large task binary with size 1357.3 KiB


In [28]:
df_test_ID.show()

+----------+--------------+----------+----------+
|      item|          user|item_index|user_index|
+----------+--------------+----------+----------+
|B0015FRC32|A28QKOPBDPSHE5|   10076.0|   10890.0|
|B000EUKRY0|A1PTTEYFE49BQM|     104.0|    6406.0|
|B007XPLI56|A2C5VTBNC6I5MY|     404.0|   15278.0|
|B003A5RTHO|A12IOCD2A7OC7K|    6845.0|    2525.0|
|B006HCVT5A|A3ISFBZ5UFK81I|    3045.0|   16376.0|
|B004GXIDYM|A34BONVNM07TRG|    2537.0|   18436.0|
|B004R1ZUNA| A3Y0IB3VYLD6A|     297.0|    5296.0|
|B0006IRTU0|A2CYXQOAR1EJRQ|     471.0|   18105.0|
|B00A88EPCI|A3L249C56OJI7D|     838.0|    8140.0|
|B00428LJ06|A1MLBMJSFK6BIJ|     142.0|    6338.0|
|B0002BSTY6|A194UXXKM11698|    7919.0|    1160.0|
|B000IBPD76| ARSNAGZWXP7GN|      39.0|   13708.0|
|B000TK8440|A35B72PSA30R67|     183.0|    1699.0|
|B000EQGT00|A304ILYRZ145SI|     135.0|     710.0|
|B00005YVRN|A3118YKNMNAS33|     134.0|    2209.0|
|B0027FFMBS|A3LHE5MHDF7X2R|   11442.0|   12545.0|
|B00AAPHZVW| ANOST6C92T7HB|     557.0|   18970.0|


22/12/31 00:17:41 WARN DAGScheduler: Broadcasting large task binary with size 1185.5 KiB


In [29]:
testall_predictions=model.transform(df_test_ID)

In [30]:
testall_predictions.printSchema()

root
 |-- item: string (nullable = true)
 |-- user: string (nullable = true)
 |-- item_index: double (nullable = false)
 |-- user_index: double (nullable = false)
 |-- prediction: float (nullable = false)



In [31]:
testall_predictions.describe().show()

22/12/31 00:17:42 WARN DAGScheduler: Broadcasting large task binary with size 1400.4 KiB
22/12/31 00:17:42 WARN DAGScheduler: Broadcasting large task binary with size 1345.7 KiB
22/12/31 00:17:42 WARN DAGScheduler: Broadcasting large task binary with size 1401.7 KiB
22/12/31 00:17:42 WARN DAGScheduler: Broadcasting large task binary with size 1575.4 KiB
22/12/31 00:17:45 WARN DAGScheduler: Broadcasting large task binary with size 1602.5 KiB


+-------+--------------------+--------------------+------------------+-----------------+----------+
|summary|                item|                user|        item_index|       user_index|prediction|
+-------+--------------------+--------------------+------------------+-----------------+----------+
|  count|               83799|               83799|             83799|            83799|     83799|
|   mean|2.1813556526264625E9|                null|3844.3237150801324|8276.219012160049|       NaN|
| stddev|2.1882518602245984E9|                null|3647.8006412185123|6463.700444398743|       NaN|
|    min|          0439893577|A012468118FTQAINE...|               0.0|              0.0|0.07626182|
|    max|          B00LBI9BKA|       AZZYW4YOE1B6E|           11824.0|          19127.0|       NaN|
+-------+--------------------+--------------------+------------------+-----------------+----------+



22/12/31 00:17:47 WARN DAGScheduler: Broadcasting large task binary with size 1598.6 KiB


In [32]:
testall_predictions.show()

22/12/31 00:17:48 WARN DAGScheduler: Broadcasting large task binary with size 1401.7 KiB
22/12/31 00:17:48 WARN DAGScheduler: Broadcasting large task binary with size 1345.7 KiB
22/12/31 00:17:48 WARN DAGScheduler: Broadcasting large task binary with size 1400.4 KiB
22/12/31 00:17:48 WARN DAGScheduler: Broadcasting large task binary with size 1575.6 KiB


+----------+--------------+----------+----------+----------+
|      item|          user|item_index|user_index|prediction|
+----------+--------------+----------+----------+----------+
|B000SQTJJO|A31BD4RXCON7QO|     148.0|    7634.0| 3.4450731|
|B000SQTJJO|A2TTHN1UMO82VY|     148.0|     329.0| 3.5525923|
|B000SQTJJO|A3LUYUZNKG378S|     148.0|   12556.0| 4.1586866|
|B000SQTJJO|A136LRUR0RDH9J|     148.0|     416.0| 3.5624144|
|B000SQTJJO|A1O402ZP60PD04|     148.0|   14618.0|  3.698164|
|B000SQTJJO|A186VDAIOH0CR1|     148.0|    9654.0| 3.9981563|
|B000SQTJJO|A1RF9YK4BK5TRH|     148.0|     206.0|  3.292471|
|B000SQTJJO|A30H2335OM7RD6|     148.0|       6.0|  4.083807|
|B000SQTJJO|A265B1IZE5RVG6|     148.0|     429.0| 3.5757942|
|B000SQTJJO| AOYAZN4MA0TVU|     148.0|    2458.0|  4.139878|
|B000SQTJJO| A2W34ZSDBOPC6|     148.0|   11665.0|  4.175356|
|B000SQTJJO|A12PN254PTSGU5|     148.0|     197.0|  3.730764|
|B000SQTJJO|A3DYBTW1TEZL3M|     148.0|    1725.0| 4.1937737|
|B000SQTJJO|A2HELIKP5RV2

22/12/31 00:17:51 WARN DAGScheduler: Broadcasting large task binary with size 1588.8 KiB


Useful function: concate User Item to generate U_I column

In [33]:
from pyspark.sql.functions import concat, col, lit

out = testall_predictions.select(concat(col("user"), lit("_"), col("item")) , testall_predictions.prediction)

In [34]:
out.printSchema()

root
 |-- concat(user, _, item): string (nullable = true)
 |-- prediction: float (nullable = false)



In [35]:
out.show()

22/12/31 00:17:52 WARN DAGScheduler: Broadcasting large task binary with size 1400.4 KiB
22/12/31 00:17:52 WARN DAGScheduler: Broadcasting large task binary with size 1401.7 KiB
22/12/31 00:17:52 WARN DAGScheduler: Broadcasting large task binary with size 1345.7 KiB
22/12/31 00:17:52 WARN DAGScheduler: Broadcasting large task binary with size 1575.5 KiB
22/12/31 00:17:55 WARN DAGScheduler: Broadcasting large task binary with size 1588.7 KiB


+---------------------+----------+
|concat(user, _, item)|prediction|
+---------------------+----------+
| A31BD4RXCON7QO_B0...| 3.4450731|
| A2TTHN1UMO82VY_B0...| 3.5525923|
| A3LUYUZNKG378S_B0...| 4.1586866|
| A136LRUR0RDH9J_B0...| 3.5624144|
| A1O402ZP60PD04_B0...|  3.698164|
| A186VDAIOH0CR1_B0...| 3.9981563|
| A1RF9YK4BK5TRH_B0...|  3.292471|
| A30H2335OM7RD6_B0...|  4.083807|
| A265B1IZE5RVG6_B0...| 3.5757942|
| AOYAZN4MA0TVU_B00...|  4.139878|
| A2W34ZSDBOPC6_B00...|  4.175356|
| A12PN254PTSGU5_B0...|  3.730764|
| A3DYBTW1TEZL3M_B0...| 4.1937737|
| A2HELIKP5RV27F_B0...| 3.5518086|
| A357B3PUHSVQA_B00...|   3.11953|
| A3J2YU2D9BH2J7_B0...|  3.588092|
| A1YCWZWOXLUAY5_B0...|       NaN|
| AQFFA5JFDLQRS_B00...| 2.9760876|
| A2H7Y21WEYO3JQ_B0...| 3.1872535|
| A1WSM2KPKNHC44_B0...|       NaN|
+---------------------+----------+
only showing top 20 rows



Remember to fix NaN value

In [36]:
out = out.na.fill(value=NAN)

In [37]:
out.show()

22/12/31 00:17:55 WARN DAGScheduler: Broadcasting large task binary with size 1345.7 KiB
22/12/31 00:17:55 WARN DAGScheduler: Broadcasting large task binary with size 1401.7 KiB
22/12/31 00:17:55 WARN DAGScheduler: Broadcasting large task binary with size 1400.4 KiB
22/12/31 00:17:56 WARN DAGScheduler: Broadcasting large task binary with size 1575.5 KiB


+---------------------+----------+
|concat(user, _, item)|prediction|
+---------------------+----------+
| A31BD4RXCON7QO_B0...| 3.4450731|
| A2TTHN1UMO82VY_B0...| 3.5525923|
| A3LUYUZNKG378S_B0...| 4.1586866|
| A136LRUR0RDH9J_B0...| 3.5624144|
| A1O402ZP60PD04_B0...|  3.698164|
| A186VDAIOH0CR1_B0...| 3.9981563|
| A1RF9YK4BK5TRH_B0...|  3.292471|
| A30H2335OM7RD6_B0...|  4.083807|
| A265B1IZE5RVG6_B0...| 3.5757942|
| AOYAZN4MA0TVU_B00...|  4.139878|
| A2W34ZSDBOPC6_B00...|  4.175356|
| A12PN254PTSGU5_B0...|  3.730764|
| A3DYBTW1TEZL3M_B0...| 4.1937737|
| A2HELIKP5RV27F_B0...| 3.5518086|
| A357B3PUHSVQA_B00...|   3.11953|
| A3J2YU2D9BH2J7_B0...|  3.588092|
| A1YCWZWOXLUAY5_B0...|       4.0|
| AQFFA5JFDLQRS_B00...| 2.9760876|
| A2H7Y21WEYO3JQ_B0...| 3.1872535|
| A1WSM2KPKNHC44_B0...|       4.0|
+---------------------+----------+
only showing top 20 rows



22/12/31 00:17:59 WARN DAGScheduler: Broadcasting large task binary with size 1589.3 KiB


Remember to sort by U_I before output

In [38]:
out.orderBy("concat(user, _, item)").show()

22/12/31 00:17:59 WARN DAGScheduler: Broadcasting large task binary with size 1400.4 KiB
22/12/31 00:17:59 WARN DAGScheduler: Broadcasting large task binary with size 1345.7 KiB
22/12/31 00:17:59 WARN DAGScheduler: Broadcasting large task binary with size 1401.7 KiB
22/12/31 00:17:59 WARN DAGScheduler: Broadcasting large task binary with size 1575.5 KiB
22/12/31 00:18:02 WARN DAGScheduler: Broadcasting large task binary with size 1590.5 KiB


+---------------------+----------+
|concat(user, _, item)|prediction|
+---------------------+----------+
| A012468118FTQAINE...|  4.898326|
| A012468118FTQAINE...|  4.379926|
| A012468118FTQAINE...|  4.422153|
| A0182108CPDLPRCXQ...| 4.1164804|
| A0182108CPDLPRCXQ...|  4.370185|
| A0182108CPDLPRCXQ...|   2.74712|
| A026961431MGW0616...| 3.0008144|
| A026961431MGW0616...| 4.5930495|
| A026961431MGW0616...| 4.5739236|
| A034597326Z83X79S...| 4.3332458|
| A034597326Z83X79S...| 3.7739987|
| A034597326Z83X79S...|  4.145909|
| A034597326Z83X79S...|  4.499639|
| A04295422T2ZG087R...| 2.0688186|
| A04295422T2ZG087R...|  3.295319|
| A060131923OZAPX4N...| 3.2827764|
| A060131923OZAPX4N...| 3.4026582|
| A060131923OZAPX4N...|  3.280408|
| A060131923OZAPX4N...| 3.6716383|
| A060131923OZAPX4N...| 2.9511797|
+---------------------+----------+
only showing top 20 rows



In [39]:
out.withColumnRenamed('concat(user, _, item)', 'U_I').withColumnRenamed('prediction', 'rating').show()

22/12/31 00:18:03 WARN DAGScheduler: Broadcasting large task binary with size 1400.4 KiB
22/12/31 00:18:03 WARN DAGScheduler: Broadcasting large task binary with size 1401.7 KiB
22/12/31 00:18:03 WARN DAGScheduler: Broadcasting large task binary with size 1345.7 KiB
22/12/31 00:18:04 WARN DAGScheduler: Broadcasting large task binary with size 1575.5 KiB


+--------------------+---------+
|                 U_I|   rating|
+--------------------+---------+
|A31BD4RXCON7QO_B0...|3.4450731|
|A2TTHN1UMO82VY_B0...|3.5525923|
|A3LUYUZNKG378S_B0...|4.1586866|
|A136LRUR0RDH9J_B0...|3.5624144|
|A1O402ZP60PD04_B0...| 3.698164|
|A186VDAIOH0CR1_B0...|3.9981563|
|A1RF9YK4BK5TRH_B0...| 3.292471|
|A30H2335OM7RD6_B0...| 4.083807|
|A265B1IZE5RVG6_B0...|3.5757942|
|AOYAZN4MA0TVU_B00...| 4.139878|
|A2W34ZSDBOPC6_B00...| 4.175356|
|A12PN254PTSGU5_B0...| 3.730764|
|A3DYBTW1TEZL3M_B0...|4.1937737|
|A2HELIKP5RV27F_B0...|3.5518086|
|A357B3PUHSVQA_B00...|  3.11953|
|A3J2YU2D9BH2J7_B0...| 3.588092|
|A1YCWZWOXLUAY5_B0...|      4.0|
|AQFFA5JFDLQRS_B00...|2.9760876|
|A2H7Y21WEYO3JQ_B0...|3.1872535|
|A1WSM2KPKNHC44_B0...|      4.0|
+--------------------+---------+
only showing top 20 rows



22/12/31 00:18:07 WARN DAGScheduler: Broadcasting large task binary with size 1589.3 KiB


In [40]:
out.withColumnRenamed('concat(user, _, item)', 'U_I').withColumnRenamed('prediction', 'rating').orderBy("U_I").coalesce(1).write.format("csv").mode('overwrite').save("./sample_submission.csv", header='true')

22/12/31 00:18:07 WARN DAGScheduler: Broadcasting large task binary with size 1400.4 KiB
22/12/31 00:18:07 WARN DAGScheduler: Broadcasting large task binary with size 1401.7 KiB
22/12/31 00:18:07 WARN DAGScheduler: Broadcasting large task binary with size 1345.7 KiB
22/12/31 00:18:08 WARN DAGScheduler: Broadcasting large task binary with size 1575.5 KiB
22/12/31 00:18:10 WARN DAGScheduler: Broadcasting large task binary with size 1590.5 KiB
22/12/31 00:18:12 WARN DAGScheduler: Broadcasting large task binary with size 1602.9 KiB
22/12/31 00:18:14 WARN DAGScheduler: Broadcasting large task binary with size 1728.3 KiB
